# <center> Proyek UAS Kapita Selekta Statistika

---

**Kelompok 10 - Butuh Libur**

6161901066 - Dennis Imanuel\
6161901104 - Putu Lila Kusumayani \
6161901114 - Bilqis Aulia R.\
6161901116 - Tiara Alamanda


# Prepare SPARK

In [ ]:
# Instalasi pyspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=1e0aff292eeed3473c1365cce8a6f2a9d3a8eb5ea3265703ae8df968e6e5ed04
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# import library PySpark
import requests
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType
from pyspark.sql.functions import avg, min, max, stddev, desc
from pyspark.sql.functions import col

In [ ]:
# Membuat spark session
spark = SparkSession.builder.appName("Analisis Skor UTBK Sosilogi").getOrCreate()

# Pipeline ETL

### Extract Datasets from Kaggle
Pastikan bahwa telah mengupload File Kaggle API Token (dari menu kaggle.com/settings/account) account masing-masing terlebih dahulu yang didapatkan dengan cara "create new token".

NB: Pastikan File kaggle.json yang diupload adalah file token yang terbaru dan masih aktif

In [ ]:
# Perintah untuk download
!pip install kaggle

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d ekojsalim/indonesia-college-entrance-examination-utbk-2019

  0% 0.00/3.75M [00:00<?, ?B/s]
100% 3.75M/3.75M [00:00<00:00, 207MB/s]


In [ ]:
!unzip indonesia-college-entrance-examination-utbk-2019

Archive:  indonesia-college-entrance-examination-utbk-2019.zip
  inflating: majors.csv              
  inflating: score_humanities.csv    
  inflating: score_science.csv       
  inflating: universities.csv        


In [ ]:
universitas = pd.read_csv("universities.csv")
urutan = universitas.columns[0]
universitas = universitas.drop(columns=[urutan])
universitas

,id_university,university_name
0,111,UNIVERSITAS SYIAH KUALA
1,112,UNIVERSITAS MALIKUSSALEH
2,113,UNIVERSITAS TEUKU UMAR
3,114,UNIVERSITAS SAMUDRA
4,115,ISBI ACEH
...,...,...
80,821,UNIVERSITAS KHAIRUN
81,911,UNIVERSITAS CENDERAWASIH
82,912,UNIVERSITAS MUSAMUS MERAUKE
83,913,ISBI TANAH PAPUA


In [ ]:
jurusan = pd.read_csv("majors.csv")
urutan_jurusan = jurusan.columns[0]
jurusan = jurusan.drop(columns=[urutan_jurusan])
jurusan

,id_major,id_university,type,major_name,capacity
0,1111014,111,science,PENDIDIKAN DOKTER HEWAN,88
1,1111022,111,science,TEKNIK SIPIL,64
2,1111037,111,science,TEKNIK MESIN,48
3,1111045,111,science,TEKNIK KIMIA,48
4,1111053,111,science,ARSITEKTUR,48
...,...,...,...,...,...
3162,9212011,921,humanities,EKONOMI PEMBANGUNAN,32
3163,9212042,921,humanities,MANAJEMEN,32
3164,9212057,921,humanities,AKUNTANSI,32
3165,9212065,921,humanities,PENDIDIKAN BAHASA INDONESIA,16


In [ ]:
# Melakukan proses extract
#data = response.json()['features']
skor_ips = pd.read_csv("score_humanities.csv")
skor_ips

,Unnamed: 0,id_first_major,id_first_university,id_second_major,id_second_university,id_user,score_eko,score_geo,score_kmb,score_kpu,score_kua,score_mat,score_ppu,score_sej,score_sos
0,0,3322014,332,3532154,353,26,778,486,679,594,643,686,559,392,676
1,1,3212057,321,3322022,332,35,579,631,710,700,810,656,638,535,686
2,2,3722057,372,1912042,191,44,600,481,314,432,522,494,461,467,574
3,3,3212081,321,3212301,321,55,559,563,646,638,591,538,585,647,589
4,4,3812106,381,3852034,385,171,838,734,654,613,690,315,628,660,674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61197,61197,3552127,355,6212115,621,328356,521,497,498,522,573,575,541,599,459
61198,61198,3512053,351,3532057,353,328357,638,464,548,576,541,492,548,435,459
61199,61199,6212115,621,6212107,621,328364,491,601,567,486,492,476,518,409,434
61200,61200,3812056,381,3822012,382,328366,573,544,634,672,527,526,504,696,463


In [ ]:
skor_ipa = pd.read_csv("score_science.csv")
skor_ipa

,Unnamed: 0,id_first_major,id_first_university,id_second_major,id_second_university,id_user,score_bio,score_fis,score_kim,score_kmb,score_kpu,score_kua,score_mat,score_ppu
0,0,3321065,332,3331187,333,4,400,400,400,400,400,400,400,400
1,1,3211015,321,3611066,361,14,816,666,651,678,685,706,695,562
2,2,3721093,372,3551302,355,19,562,839,624,700,781,464,551,668
3,3,3321096,332,3551194,355,23,700,669,692,679,692,813,507,573
4,4,5211104,521,5211085,521,28,461,619,441,593,563,500,666,370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86565,86565,1411161,141,1421147,142,344111,516,412,376,546,523,490,549,520
86566,86566,3561262,356,3571055,357,344125,334,634,435,613,461,390,399,554
86567,86567,3531182,353,3561022,356,344127,316,717,421,474,494,483,327,554
86568,86568,1121094,112,1221095,122,344151,592,451,583,437,447,485,508,492


### Transform Datasets
Tujuan: menggabungkan data tabel "skor_ips" dan "skor_ipa" dengan tabel "universitas" dan " jurusan" menjadi satu dataframe berdasarkan hubungan (*connection*) ID_universitas dan ID_major.

#### IPS

In [ ]:
urutan_skor = skor_ips.columns[0]
skor_ips = skor_ips.drop(columns=[urutan_skor])
skor_ips

,id_first_major,id_first_university,id_second_major,id_second_university,id_user,score_eko,score_geo,score_kmb,score_kpu,score_kua,score_mat,score_ppu,score_sej,score_sos
0,3322014,332,3532154,353,26,778,486,679,594,643,686,559,392,676
1,3212057,321,3322022,332,35,579,631,710,700,810,656,638,535,686
2,3722057,372,1912042,191,44,600,481,314,432,522,494,461,467,574
3,3212081,321,3212301,321,55,559,563,646,638,591,538,585,647,589
4,3812106,381,3852034,385,171,838,734,654,613,690,315,628,660,674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61197,3552127,355,6212115,621,328356,521,497,498,522,573,575,541,599,459
61198,3512053,351,3532057,353,328357,638,464,548,576,541,492,548,435,459
61199,6212115,621,6212107,621,328364,491,601,567,486,492,476,518,409,434
61200,3812056,381,3822012,382,328366,573,544,634,672,527,526,504,696,463


In [ ]:
merged_ips = pd.merge(skor_ips, universitas, left_on='id_first_university', right_on='id_university')
merged_ips1 = pd.merge(merged_ips, universitas, left_on='id_second_university', right_on='id_university')
merged_ips1

,id_first_major,id_first_university,id_second_major,id_second_university,id_user,score_eko,score_geo,score_kmb,score_kpu,score_kua,score_mat,score_ppu,score_sej,score_sos,id_university_x,university_name_x,id_university_y,university_name_y
0,3322014,332,3532154,353,26,778,486,679,594,643,686,559,392,676,332,INSTITUT TEKNOLOGI BANDUNG,353,UNIVERSITAS SEBELAS MARET
1,3322014,332,3532146,353,7203,623,570,531,544,513,490,526,464,579,332,INSTITUT TEKNOLOGI BANDUNG,353,UNIVERSITAS SEBELAS MARET
2,3322022,332,3532115,353,9868,625,562,528,662,380,323,704,553,529,332,INSTITUT TEKNOLOGI BANDUNG,353,UNIVERSITAS SEBELAS MARET
3,3322022,332,3532026,353,9933,614,578,629,585,488,432,608,518,747,332,INSTITUT TEKNOLOGI BANDUNG,353,UNIVERSITAS SEBELAS MARET
4,3322014,332,3532042,353,13427,669,642,636,685,681,608,644,573,656,332,INSTITUT TEKNOLOGI BANDUNG,353,UNIVERSITAS SEBELAS MARET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61193,9212057,921,9122043,912,17088,391,465,591,646,463,543,530,628,446,921,UNIVERSITAS PAPUA,912,UNIVERSITAS MUSAMUS MERAUKE
61194,9212042,921,9122027,912,198567,460,634,593,498,654,614,633,711,486,921,UNIVERSITAS PAPUA,912,UNIVERSITAS MUSAMUS MERAUKE
61195,7532096,753,9122066,912,259647,548,414,415,501,507,440,475,519,336,753,UNIVERSITAS SEMBILAN BELAS NOVEMBER KOLAKA,912,UNIVERSITAS MUSAMUS MERAUKE
61196,9122027,912,9122027,912,202832,672,604,487,564,546,500,463,643,521,912,UNIVERSITAS MUSAMUS MERAUKE,912,UNIVERSITAS MUSAMUS MERAUKE


In [ ]:
ips_univ = merged_ips1[['id_user','id_first_university','university_name_x','id_first_major','id_second_university','university_name_y','id_second_major', 'score_eko', 'score_geo', 'score_kmb', 'score_kpu', 'score_kua', 'score_mat', 'score_ppu', 'score_sej', 'score_sos']]
ips_univ

,id_user,id_first_university,university_name_x,id_first_major,id_second_university,university_name_y,id_second_major,score_eko,score_geo,score_kmb,score_kpu,score_kua,score_mat,score_ppu,score_sej,score_sos
0,26,332,INSTITUT TEKNOLOGI BANDUNG,3322014,353,UNIVERSITAS SEBELAS MARET,3532154,778,486,679,594,643,686,559,392,676
1,7203,332,INSTITUT TEKNOLOGI BANDUNG,3322014,353,UNIVERSITAS SEBELAS MARET,3532146,623,570,531,544,513,490,526,464,579
2,9868,332,INSTITUT TEKNOLOGI BANDUNG,3322022,353,UNIVERSITAS SEBELAS MARET,3532115,625,562,528,662,380,323,704,553,529
3,9933,332,INSTITUT TEKNOLOGI BANDUNG,3322022,353,UNIVERSITAS SEBELAS MARET,3532026,614,578,629,585,488,432,608,518,747
4,13427,332,INSTITUT TEKNOLOGI BANDUNG,3322014,353,UNIVERSITAS SEBELAS MARET,3532042,669,642,636,685,681,608,644,573,656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61193,17088,921,UNIVERSITAS PAPUA,9212057,912,UNIVERSITAS MUSAMUS MERAUKE,9122043,391,465,591,646,463,543,530,628,446
61194,198567,921,UNIVERSITAS PAPUA,9212042,912,UNIVERSITAS MUSAMUS MERAUKE,9122027,460,634,593,498,654,614,633,711,486
61195,259647,753,UNIVERSITAS SEMBILAN BELAS NOVEMBER KOLAKA,7532096,912,UNIVERSITAS MUSAMUS MERAUKE,9122066,548,414,415,501,507,440,475,519,336
61196,202832,912,UNIVERSITAS MUSAMUS MERAUKE,9122027,912,UNIVERSITAS MUSAMUS MERAUKE,9122027,672,604,487,564,546,500,463,643,521


In [ ]:
merged_ips2 = pd.merge(ips_univ, jurusan, left_on='id_first_major', right_on='id_major')
merged_ips3 = pd.merge(merged_ips2, jurusan, left_on='id_second_major', right_on='id_major')
merged_ips3

,id_user,id_first_university,university_name_x,id_first_major,id_second_university,university_name_y,id_second_major,score_eko,score_geo,score_kmb,...,id_major_x,id_university_x,type_x,major_name_x,capacity_x,id_major_y,id_university_y,type_y,major_name_y,capacity_y
0,26,332,INSTITUT TEKNOLOGI BANDUNG,3322014,353,UNIVERSITAS SEBELAS MARET,3532154,778,486,679,...,3322014,332,humanities,FAKULTAS SENIRUPA DAN DESAIN (FSRD),100,3532154,353,humanities,DESAIN KOMUNIKASI VISUAL,30
1,15713,332,INSTITUT TEKNOLOGI BANDUNG,3322014,353,UNIVERSITAS SEBELAS MARET,3532154,803,638,659,...,3322014,332,humanities,FAKULTAS SENIRUPA DAN DESAIN (FSRD),100,3532154,353,humanities,DESAIN KOMUNIKASI VISUAL,30
2,24011,332,INSTITUT TEKNOLOGI BANDUNG,3322014,353,UNIVERSITAS SEBELAS MARET,3532154,601,560,575,...,3322014,332,humanities,FAKULTAS SENIRUPA DAN DESAIN (FSRD),100,3532154,353,humanities,DESAIN KOMUNIKASI VISUAL,30
3,25412,332,INSTITUT TEKNOLOGI BANDUNG,3322014,353,UNIVERSITAS SEBELAS MARET,3532154,612,494,524,...,3322014,332,humanities,FAKULTAS SENIRUPA DAN DESAIN (FSRD),100,3532154,353,humanities,DESAIN KOMUNIKASI VISUAL,30
4,37291,332,INSTITUT TEKNOLOGI BANDUNG,3322014,353,UNIVERSITAS SEBELAS MARET,3532154,493,573,586,...,3322014,332,humanities,FAKULTAS SENIRUPA DAN DESAIN (FSRD),100,3532154,353,humanities,DESAIN KOMUNIKASI VISUAL,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61193,298546,752,UNIVERSITAS NEGERI GORONTALO,7522047,752,UNIVERSITAS NEGERI GORONTALO,7522167,469,541,578,...,7522047,752,humanities,PENDIDIKAN JASMANI KESEHATAN DAN REKREASI,90,7522167,752,humanities,PENDIDIKAN KEPELATIHAN,54
61194,221842,511,UNIVERSITAS TANJUNGPURA,5112114,722,UNIVERSITAS NEGERI MANADO,7222276,411,439,479,...,5112114,511,humanities,PEND. GURU PEND. ANAK USIA DINI (PGPAUD),20,7222276,722,humanities,PEND. ANAK USIA DINI,32
61195,187531,751,UNIVERSITAS HALUOLEO,7512215,753,UNIVERSITAS SEMBILAN BELAS NOVEMBER KOLAKA,7532034,421,315,477,...,7512215,751,humanities,ILMU KESEJAHTERAAN SOSIAL,48,7532034,753,humanities,PENDIDIKAN BAHASA INDONESIA,64
61196,326280,821,UNIVERSITAS KHAIRUN,8212131,821,UNIVERSITAS KHAIRUN,8212026,465,506,308,...,8212131,821,humanities,ANTROPOLOGI SOSIAL,16,8212026,821,humanities,EKONOMI PEMBANGUNAN,60


In [ ]:
ips_univ_jurusan = merged_ips3[['id_user','id_first_university','university_name_x','id_first_major','major_name_x','capacity_x','id_second_university','university_name_y','id_second_major','major_name_y','capacity_y', 'score_eko', 'score_geo', 'score_kmb', 'score_kpu', 'score_kua', 'score_mat', 'score_ppu', 'score_sej', 'score_sos']]
#ips_univ_jurusan = merged_univ_jurusan.rename(columns = {'id_first_university':'id_univ_pil_1', 'university_name_x':'univ_pil_1', 'id_first_major':'id_jurusan_pil_1', 'major_name_x':'jurusan_pil_1', 'capacity_x':'kapasitas_pil_1', 'id_second_university':'id_univ_pil2', 'university_name_y':'univ_pil_2', 'id_second_major':'id_jurusan_pil_1', 'major_name_y':'jurusan_pil_2', 'capacity_y':'kapasitas_pil_2'}, inplace = True)
#ips_univ_jurusan
ips_univ_jurusan['score_utbk'] = (ips_univ_jurusan['score_eko']+ips_univ_jurusan['score_geo']+ips_univ_jurusan['score_kmb']+ips_univ_jurusan['score_kpu']+ips_univ_jurusan['score_kua']+ips_univ_jurusan['score_mat']+ips_univ_jurusan['score_ppu']+ips_univ_jurusan['score_sej']+ips_univ_jurusan['score_sos'])/9
ips_univ_jurusan['score_utbk'] = ips_univ_jurusan['score_utbk'].astype(int)
ips_univ_jurusan

<ipython-input-20-e489a2b129ca>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ips_univ_jurusan['score_utbk'] = (ips_univ_jurusan['score_eko']+ips_univ_jurusan['score_geo']+ips_univ_jurusan['score_kmb']+ips_univ_jurusan['score_kpu']+ips_univ_jurusan['score_kua']+ips_univ_jurusan['score_mat']+ips_univ_jurusan['score_ppu']+ips_univ_jurusan['score_sej']+ips_univ_jurusan['score_sos'])/9
<ipython-input-20-e489a2b129ca>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ips_univ_jurusan['score_utbk'] = ips_univ_j

,id_user,id_first_university,university_name_x,id_first_major,major_name_x,capacity_x,id_second_university,university_name_y,id_second_major,major_name_y,...,score_eko,score_geo,score_kmb,score_kpu,score_kua,score_mat,score_ppu,score_sej,score_sos,score_utbk
0,26,332,INSTITUT TEKNOLOGI BANDUNG,3322014,FAKULTAS SENIRUPA DAN DESAIN (FSRD),100,353,UNIVERSITAS SEBELAS MARET,3532154,DESAIN KOMUNIKASI VISUAL,...,778,486,679,594,643,686,559,392,676,610
1,15713,332,INSTITUT TEKNOLOGI BANDUNG,3322014,FAKULTAS SENIRUPA DAN DESAIN (FSRD),100,353,UNIVERSITAS SEBELAS MARET,3532154,DESAIN KOMUNIKASI VISUAL,...,803,638,659,696,685,494,576,676,694,657
2,24011,332,INSTITUT TEKNOLOGI BANDUNG,3322014,FAKULTAS SENIRUPA DAN DESAIN (FSRD),100,353,UNIVERSITAS SEBELAS MARET,3532154,DESAIN KOMUNIKASI VISUAL,...,601,560,575,611,629,509,607,775,629,610
3,25412,332,INSTITUT TEKNOLOGI BANDUNG,3322014,FAKULTAS SENIRUPA DAN DESAIN (FSRD),100,353,UNIVERSITAS SEBELAS MARET,3532154,DESAIN KOMUNIKASI VISUAL,...,612,494,524,549,589,548,508,617,548,554
4,37291,332,INSTITUT TEKNOLOGI BANDUNG,3322014,FAKULTAS SENIRUPA DAN DESAIN (FSRD),100,353,UNIVERSITAS SEBELAS MARET,3532154,DESAIN KOMUNIKASI VISUAL,...,493,573,586,582,504,641,608,474,560,557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61193,298546,752,UNIVERSITAS NEGERI GORONTALO,7522047,PENDIDIKAN JASMANI KESEHATAN DAN REKREASI,90,752,UNIVERSITAS NEGERI GORONTALO,7522167,PENDIDIKAN KEPELATIHAN,...,469,541,578,416,446,618,331,572,435,489
61194,221842,511,UNIVERSITAS TANJUNGPURA,5112114,PEND. GURU PEND. ANAK USIA DINI (PGPAUD),20,722,UNIVERSITAS NEGERI MANADO,7222276,PEND. ANAK USIA DINI,...,411,439,479,371,508,613,443,485,561,478
61195,187531,751,UNIVERSITAS HALUOLEO,7512215,ILMU KESEJAHTERAAN SOSIAL,48,753,UNIVERSITAS SEMBILAN BELAS NOVEMBER KOLAKA,7532034,PENDIDIKAN BAHASA INDONESIA,...,421,315,477,537,553,443,460,452,535,465
61196,326280,821,UNIVERSITAS KHAIRUN,8212131,ANTROPOLOGI SOSIAL,16,821,UNIVERSITAS KHAIRUN,8212026,EKONOMI PEMBANGUNAN,...,465,506,308,385,459,522,406,471,347,429


Pilihan untuk mendownload dataset IPS
yang telah ditransform

In [ ]:
#ips_univ_jurusan.to_excel("gabungan_ips.xlsx")

#### IPA

In [ ]:
urutan_skor = skor_ipa.columns[0]
skor_ipa = skor_ipa.drop(columns=[urutan_skor])
skor_ipa

,id_first_major,id_first_university,id_second_major,id_second_university,id_user,score_bio,score_fis,score_kim,score_kmb,score_kpu,score_kua,score_mat,score_ppu
0,3321065,332,3331187,333,4,400,400,400,400,400,400,400,400
1,3211015,321,3611066,361,14,816,666,651,678,685,706,695,562
2,3721093,372,3551302,355,19,562,839,624,700,781,464,551,668
3,3321096,332,3551194,355,23,700,669,692,679,692,813,507,573
4,5211104,521,5211085,521,28,461,619,441,593,563,500,666,370
...,...,...,...,...,...,...,...,...,...,...,...,...,...
86565,1411161,141,1421147,142,344111,516,412,376,546,523,490,549,520
86566,3561262,356,3571055,357,344125,334,634,435,613,461,390,399,554
86567,3531182,353,3561022,356,344127,316,717,421,474,494,483,327,554
86568,1121094,112,1221095,122,344151,592,451,583,437,447,485,508,492


In [ ]:
merged_ipa = pd.merge(skor_ipa, universitas, left_on='id_first_university', right_on='id_university')
merged_ipa1 = pd.merge(merged_ipa, universitas, left_on='id_second_university', right_on='id_university')
merged_ipa1

,id_first_major,id_first_university,id_second_major,id_second_university,id_user,score_bio,score_fis,score_kim,score_kmb,score_kpu,score_kua,score_mat,score_ppu,id_university_x,university_name_x,id_university_y,university_name_y
0,3321065,332,3331187,333,4,400,400,400,400,400,400,400,400,332,INSTITUT TEKNOLOGI BANDUNG,333,UNIVERSITAS PADJADJARAN
1,3321026,332,3331172,333,177,589,712,771,666,753,732,681,714,332,INSTITUT TEKNOLOGI BANDUNG,333,UNIVERSITAS PADJADJARAN
2,3321065,332,3331206,333,2149,546,541,686,581,631,601,615,467,332,INSTITUT TEKNOLOGI BANDUNG,333,UNIVERSITAS PADJADJARAN
3,3321011,332,3331172,333,2404,574,700,486,718,654,540,478,682,332,INSTITUT TEKNOLOGI BANDUNG,333,UNIVERSITAS PADJADJARAN
4,3321057,332,3331164,333,2637,646,467,464,651,607,577,452,601,332,INSTITUT TEKNOLOGI BANDUNG,333,UNIVERSITAS PADJADJARAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86564,7411033,741,7411056,741,305882,499,449,387,393,492,445,517,390,741,UNIVERSITAS SULAWESI BARAT,741,UNIVERSITAS SULAWESI BARAT
86565,3331102,333,3561207,354,15481,519,575,656,556,646,706,545,585,333,UNIVERSITAS PADJADJARAN,354,ISI SURAKARTA
86566,1331021,354,3421082,354,105997,499,483,611,543,592,541,733,561,354,ISI SURAKARTA,354,ISI SURAKARTA
86567,3612232,361,3642092,364,118700,505,400,326,579,674,524,489,576,361,UNIVERSITAS GADJAH MADA,364,ISI YOGYAKARTA


In [ ]:
ipa_univ = merged_ipa1[['id_user','id_first_university','university_name_x','id_first_major','id_second_university','university_name_y','id_second_major', 'score_bio', 'score_fis', 'score_kim', 'score_kmb', 'score_kpu', 'score_kua', 'score_mat', 'score_ppu']]
ipa_univ

,id_user,id_first_university,university_name_x,id_first_major,id_second_university,university_name_y,id_second_major,score_bio,score_fis,score_kim,score_kmb,score_kpu,score_kua,score_mat,score_ppu
0,4,332,INSTITUT TEKNOLOGI BANDUNG,3321065,333,UNIVERSITAS PADJADJARAN,3331187,400,400,400,400,400,400,400,400
1,177,332,INSTITUT TEKNOLOGI BANDUNG,3321026,333,UNIVERSITAS PADJADJARAN,3331172,589,712,771,666,753,732,681,714
2,2149,332,INSTITUT TEKNOLOGI BANDUNG,3321065,333,UNIVERSITAS PADJADJARAN,3331206,546,541,686,581,631,601,615,467
3,2404,332,INSTITUT TEKNOLOGI BANDUNG,3321011,333,UNIVERSITAS PADJADJARAN,3331172,574,700,486,718,654,540,478,682
4,2637,332,INSTITUT TEKNOLOGI BANDUNG,3321057,333,UNIVERSITAS PADJADJARAN,3331164,646,467,464,651,607,577,452,601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86564,305882,741,UNIVERSITAS SULAWESI BARAT,7411033,741,UNIVERSITAS SULAWESI BARAT,7411056,499,449,387,393,492,445,517,390
86565,15481,333,UNIVERSITAS PADJADJARAN,3331102,354,ISI SURAKARTA,3561207,519,575,656,556,646,706,545,585
86566,105997,354,ISI SURAKARTA,1331021,354,ISI SURAKARTA,3421082,499,483,611,543,592,541,733,561
86567,118700,361,UNIVERSITAS GADJAH MADA,3612232,364,ISI YOGYAKARTA,3642092,505,400,326,579,674,524,489,576


In [ ]:
merged_ipa2 = pd.merge(ipa_univ, jurusan, left_on='id_first_major', right_on='id_major')
merged_ipa3 = pd.merge(merged_ipa2, jurusan, left_on='id_second_major', right_on='id_major')
merged_ipa3

,id_user,id_first_university,university_name_x,id_first_major,id_second_university,university_name_y,id_second_major,score_bio,score_fis,score_kim,...,id_major_x,id_university_x,type_x,major_name_x,capacity_x,id_major_y,id_university_y,type_y,major_name_y,capacity_y
0,4,332,INSTITUT TEKNOLOGI BANDUNG,3321065,333,UNIVERSITAS PADJADJARAN,3331187,400,400,400,...,3321065,332,science,SEKOLAH ILMU & TEKNO. HAYATI - PROG. SAINS,64,3331187,333,science,PSIKOLOGI,80
1,43597,332,INSTITUT TEKNOLOGI BANDUNG,3321065,333,UNIVERSITAS PADJADJARAN,3331187,704,447,630,...,3321065,332,science,SEKOLAH ILMU & TEKNO. HAYATI - PROG. SAINS,64,3331187,333,science,PSIKOLOGI,80
2,174427,332,INSTITUT TEKNOLOGI BANDUNG,3321065,333,UNIVERSITAS PADJADJARAN,3331187,715,532,633,...,3321065,332,science,SEKOLAH ILMU & TEKNO. HAYATI - PROG. SAINS,64,3331187,333,science,PSIKOLOGI,80
3,212163,332,INSTITUT TEKNOLOGI BANDUNG,3321065,333,UNIVERSITAS PADJADJARAN,3331187,446,511,400,...,3321065,332,science,SEKOLAH ILMU & TEKNO. HAYATI - PROG. SAINS,64,3331187,333,science,PSIKOLOGI,80
4,257481,332,INSTITUT TEKNOLOGI BANDUNG,3321065,333,UNIVERSITAS PADJADJARAN,3331187,489,533,367,...,3321065,332,science,SEKOLAH ILMU & TEKNO. HAYATI - PROG. SAINS,64,3331187,333,science,PSIKOLOGI,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86564,313981,521,UNIVERSITAS PALANGKARAYA,5211255,521,UNIVERSITAS PALANGKARAYA,5211247,482,516,453,...,5211255,521,science,KIMIA,30,5211247,521,science,FISIKA,30
86565,230738,752,UNIVERSITAS NEGERI GORONTALO,7521044,752,UNIVERSITAS NEGERI GORONTALO,7521141,528,383,464,...,7521044,752,science,PENDIDIKAN KIMIA,43,7521141,752,science,BUDIDAYA PERAIRAN,90
86566,284331,752,UNIVERSITAS NEGERI GORONTALO,7521052,752,UNIVERSITAS NEGERI GORONTALO,7521334,336,466,476,...,7521052,752,science,PENDIDIKAN GEOGRAFI,54,7521334,752,science,PENDIDIKAN ILMU PENGETAHUAN ALAM,27
86567,291267,551,UNIVERSITAS BORNEO TARAKAN,5511107,551,UNIVERSITAS BORNEO TARAKAN,5511034,520,483,654,...,5511107,551,science,AKUAKULTUR,48,5511034,551,science,AGRIBISNIS,48


In [ ]:
ipa_univ_jurusan = merged_ipa3[['id_user','id_first_university','university_name_x','id_first_major','major_name_x','capacity_x','id_second_university','university_name_y','id_second_major','major_name_y','capacity_y', 'score_bio', 'score_fis', 'score_kim', 'score_kmb', 'score_kpu', 'score_kua', 'score_mat', 'score_ppu']]
#ips_univ_jurusan = merged_univ_jurusan.rename(columns = {'id_first_university':'id_univ_pil_1', 'university_name_x':'univ_pil_1', 'id_first_major':'id_jurusan_pil_1', 'major_name_x':'jurusan_pil_1', 'capacity_x':'kapasitas_pil_1', 'id_second_university':'id_univ_pil2', 'university_name_y':'univ_pil_2', 'id_second_major':'id_jurusan_pil_1', 'major_name_y':'jurusan_pil_2', 'capacity_y':'kapasitas_pil_2'}, inplace = True)
#ips_univ_jurusan
ipa_univ_jurusan['score_utbk'] = (ipa_univ_jurusan['score_bio']+ipa_univ_jurusan['score_fis']+ipa_univ_jurusan['score_kim']+ipa_univ_jurusan['score_kmb']+ipa_univ_jurusan['score_kpu']+ipa_univ_jurusan['score_kua']+ipa_univ_jurusan['score_mat']+ipa_univ_jurusan['score_ppu'])/8
ipa_univ_jurusan['score_utbk'] = ipa_univ_jurusan['score_utbk'].astype(int)
ipa_univ_jurusan

<ipython-input-26-14a5f28cb981>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipa_univ_jurusan['score_utbk'] = (ipa_univ_jurusan['score_bio']+ipa_univ_jurusan['score_fis']+ipa_univ_jurusan['score_kim']+ipa_univ_jurusan['score_kmb']+ipa_univ_jurusan['score_kpu']+ipa_univ_jurusan['score_kua']+ipa_univ_jurusan['score_mat']+ipa_univ_jurusan['score_ppu'])/8
<ipython-input-26-14a5f28cb981>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipa_univ_jurusan['score_utbk'] = ipa_univ_jurusan['score_utbk'].astype(in

,id_user,id_first_university,university_name_x,id_first_major,major_name_x,capacity_x,id_second_university,university_name_y,id_second_major,major_name_y,capacity_y,score_bio,score_fis,score_kim,score_kmb,score_kpu,score_kua,score_mat,score_ppu,score_utbk
0,4,332,INSTITUT TEKNOLOGI BANDUNG,3321065,SEKOLAH ILMU & TEKNO. HAYATI - PROG. SAINS,64,333,UNIVERSITAS PADJADJARAN,3331187,PSIKOLOGI,80,400,400,400,400,400,400,400,400,400
1,43597,332,INSTITUT TEKNOLOGI BANDUNG,3321065,SEKOLAH ILMU & TEKNO. HAYATI - PROG. SAINS,64,333,UNIVERSITAS PADJADJARAN,3331187,PSIKOLOGI,80,704,447,630,561,518,541,585,599,573
2,174427,332,INSTITUT TEKNOLOGI BANDUNG,3321065,SEKOLAH ILMU & TEKNO. HAYATI - PROG. SAINS,64,333,UNIVERSITAS PADJADJARAN,3331187,PSIKOLOGI,80,715,532,633,579,732,804,402,608,625
3,212163,332,INSTITUT TEKNOLOGI BANDUNG,3321065,SEKOLAH ILMU & TEKNO. HAYATI - PROG. SAINS,64,333,UNIVERSITAS PADJADJARAN,3331187,PSIKOLOGI,80,446,511,400,548,679,567,513,538,525
4,257481,332,INSTITUT TEKNOLOGI BANDUNG,3321065,SEKOLAH ILMU & TEKNO. HAYATI - PROG. SAINS,64,333,UNIVERSITAS PADJADJARAN,3331187,PSIKOLOGI,80,489,533,367,481,487,544,499,469,483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86564,313981,521,UNIVERSITAS PALANGKARAYA,5211255,KIMIA,30,521,UNIVERSITAS PALANGKARAYA,5211247,FISIKA,30,482,516,453,546,569,597,587,633,547
86565,230738,752,UNIVERSITAS NEGERI GORONTALO,7521044,PENDIDIKAN KIMIA,43,752,UNIVERSITAS NEGERI GORONTALO,7521141,BUDIDAYA PERAIRAN,90,528,383,464,550,516,603,364,419,478
86566,284331,752,UNIVERSITAS NEGERI GORONTALO,7521052,PENDIDIKAN GEOGRAFI,54,752,UNIVERSITAS NEGERI GORONTALO,7521334,PENDIDIKAN ILMU PENGETAHUAN ALAM,27,336,466,476,511,378,393,589,470,452
86567,291267,551,UNIVERSITAS BORNEO TARAKAN,5511107,AKUAKULTUR,48,551,UNIVERSITAS BORNEO TARAKAN,5511034,AGRIBISNIS,48,520,483,654,545,606,722,425,495,556


Pilihan untuk mendownload dataset IPA yang telah ditransform

In [ ]:
#ipa_univ_jurusan.to_excel("gabungan_ipa.xlsx")

### Load to PySpark

In [ ]:
df1 = spark.createDataFrame(ips_univ_jurusan)
df1.show()

+-------+-------------------+--------------------+--------------+--------------------+----------+--------------------+--------------------+---------------+--------------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+
|id_user|id_first_university|   university_name_x|id_first_major|        major_name_x|capacity_x|id_second_university|   university_name_y|id_second_major|        major_name_y|capacity_y|score_eko|score_geo|score_kmb|score_kpu|score_kua|score_mat|score_ppu|score_sej|score_sos|score_utbk|
+-------+-------------------+--------------------+--------------+--------------------+----------+--------------------+--------------------+---------------+--------------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+
|     26|                332|INSTITUT TEKNOLOG...|       3322014|FAKULTAS SENIRUPA...|       100|                 353|UNIVERSITAS SEB

In [ ]:
df2 = spark.createDataFrame(ipa_univ_jurusan)
df2.show()

+-------+-------------------+--------------------+--------------+--------------------+----------+--------------------+--------------------+---------------+------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+----------+
|id_user|id_first_university|   university_name_x|id_first_major|        major_name_x|capacity_x|id_second_university|   university_name_y|id_second_major|major_name_y|capacity_y|score_bio|score_fis|score_kim|score_kmb|score_kpu|score_kua|score_mat|score_ppu|score_utbk|
+-------+-------------------+--------------------+--------------+--------------------+----------+--------------------+--------------------+---------------+------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+----------+
|      4|                332|INSTITUT TEKNOLOG...|       3321065|SEKOLAH ILMU & TE...|        64|                 333|UNIVERSITAS PADJA...|        3331187|   PSIKOLOGI|        80|      40

In [ ]:
df1.createOrReplaceTempView("data_ips")
df2.createOrReplaceTempView("data_ipa")

# Analysis with PySpark SQL

### 1. Jurusan paling diminati dari IPA dan IPS

#### 1a. Top 10 Jurusan paling diminati dari penjurusan IPA

In [ ]:
favorit_ipa = spark.sql("""
  SELECT Nama_Jurusan, COUNT(*) AS Jumlah_Pendaftar
FROM (
    SELECT major_name_x AS Nama_Jurusan FROM data_ipa
    UNION ALL
    SELECT major_name_y AS Nama_Jurusan FROM data_ipa
) AS combined_data
GROUP BY Nama_Jurusan
ORDER BY Jumlah_Pendaftar DESC
LIMIT 10;
"""
)
favorit_ipa.limit(favorit_ipa.count()).show(truncate=False)

+------------------+----------------+
|Nama_Jurusan      |Jumlah_Pendaftar|
+------------------+----------------+
|PENDIDIKAN DOKTER |9415            |
|TEKNIK SIPIL      |6299            |
|FARMASI           |5793            |
|TEKNIK INFORMATIKA|4889            |
|TEKNIK INDUSTRI   |4838            |
|BIOLOGI           |4516            |
|AGRIBISNIS        |4242            |
|TEKNIK ELEKTRO    |3976            |
|MATEMATIKA        |3925            |
|KIMIA             |3835            |
+------------------+----------------+



#### 1b. Top 10 Jurusan paling diminati dari penjurusan IPS

In [ ]:
favorit_ips = spark.sql("""
  SELECT Nama_Jurusan, COUNT(*) AS Jumlah_Pendaftar
FROM (
    SELECT major_name_x AS Nama_Jurusan FROM data_ips
    UNION ALL
    SELECT major_name_y AS Nama_Jurusan FROM data_ips
) AS combined_data
GROUP BY Nama_Jurusan
ORDER BY Jumlah_Pendaftar DESC
LIMIT 10;
"""
)
favorit_ips.limit(favorit_ips.count()).show(truncate=False)

+-------------------+----------------+
|Nama_Jurusan       |Jumlah_Pendaftar|
+-------------------+----------------+
|MANAJEMEN          |10492           |
|ILMU HUKUM         |8345            |
|ILMU KOMUNIKASI    |7938            |
|AKUNTANSI          |7468            |
|PSIKOLOGI          |5628            |
|EKONOMI PEMBANGUNAN|3603            |
|SOSIOLOGI          |3335            |
|HUKUM              |2797            |
|SASTRA INGGRIS     |2369            |
|ILMU POLITIK       |2131            |
+-------------------+----------------+



### 2. Top 10 Universitas Paling Diminati

In [ ]:
favorit_univ1 = spark.sql("""
  SELECT Nama_Universitas, COUNT(*) AS Jumlah_Pendaftar
FROM (
    SELECT university_name_x AS Nama_Universitas FROM data_ipa
    UNION ALL
    SELECT university_name_y AS Nama_Universitas FROM data_ipa
) AS combined_data
GROUP BY Nama_Universitas
ORDER BY Jumlah_Pendaftar DESC;
"""
).createOrReplaceTempView("favorit_univ1")

In [ ]:
favorit_univ2 = spark.sql("""
  SELECT Nama_Universitas, COUNT(*) AS Jumlah_Pendaftar
FROM (
    SELECT university_name_x AS Nama_Universitas FROM data_ips
    UNION ALL
    SELECT university_name_y AS Nama_Universitas FROM data_ips
) AS combined_data
GROUP BY Nama_Universitas
ORDER BY Jumlah_Pendaftar DESC;
"""
).createOrReplaceTempView("favorit_univ2")

In [ ]:
favorit_univ = spark.sql("""
  SELECT Nama_Universitas, SUM(Jumlah_Pendaftar) AS Jumlah_Pendaftar
  FROM (
    SELECT Nama_Universitas, Jumlah_Pendaftar
    FROM (
      SELECT favorit_univ1.Nama_Universitas, favorit_univ1.Jumlah_Pendaftar
      FROM favorit_univ1
      UNION ALL
      SELECT favorit_univ2.Nama_Universitas, favorit_univ2.Jumlah_Pendaftar
      FROM favorit_univ2
    ) AS combined_data
  ) AS subquery
  GROUP BY Nama_Universitas
  ORDER BY Jumlah_Pendaftar DESC
  LIMIT 10
""")

favorit_univ.limit(favorit_univ.count()).show(truncate=False)

+--------------------------------+----------------+
|Nama_Universitas                |Jumlah_Pendaftar|
+--------------------------------+----------------+
|UNIVERSITAS BRAWIJAYA           |18839           |
|UNIVERSITAS DIPONEGORO          |16193           |
|UNIVERSITAS PADJADJARAN         |15366           |
|UNIVERSITAS GADJAH MADA         |13836           |
|UNIVERSITAS SEBELAS MARET       |13783           |
|UNIVERSITAS INDONESIA           |13716           |
|UNIVERSITAS PENDIDIKAN INDONESIA|10427           |
|UNIVERSITAS JENDERAL SOEDIRMAN  |9156            |
|UNIVERSITAS NEGERI SEMARANG     |8617            |
|UNIVERSITAS JEMBER              |8416            |
+--------------------------------+----------------+



### 3. Berdasarkan jurusan paling diminati universitas mana yang paling banyak peminatnya?

#### 3a. Top 10 Universitas Paling Diminati untuk Pemilihan Jurusan Favorit IPA (yaitu: PENDIDIKAN DOKTER)

In [ ]:
favorit_univ_dokter = spark.sql("""
  SELECT Nama_Universitas, COUNT(*) AS Jumlah_Pendaftar
FROM (
    SELECT university_name_x AS Nama_Universitas FROM data_ipa
    UNION ALL
    SELECT university_name_y AS Nama_Universitas FROM data_ipa
    WHERE data_ipa.major_name_x = "PENDIDIKAN DOKTER" OR data_ipa.major_name_y = "PENDIDIKAN DOKTER"
) AS combined_data
GROUP BY Nama_Universitas
ORDER BY Jumlah_Pendaftar DESC
LIMIT 10;
"""
)
favorit_univ_dokter.limit(favorit_univ_dokter.count()).show(truncate=False)

+-----------------------------------+----------------+
|Nama_Universitas                   |Jumlah_Pendaftar|
+-----------------------------------+----------------+
|UNIVERSITAS BRAWIJAYA              |6611            |
|UNIVERSITAS GADJAH MADA            |5792            |
|UNIVERSITAS DIPONEGORO             |5596            |
|UNIVERSITAS PADJADJARAN            |5114            |
|UNIVERSITAS INDONESIA              |4741            |
|UNIVERSITAS SEBELAS MARET          |4105            |
|INSTITUT PERTANIAN BOGOR           |3982            |
|INSTITUT TEKNOLOGI SEPULUH NOPEMBER|3435            |
|INSTITUT TEKNOLOGI BANDUNG         |3371            |
|UNIVERSITAS JENDERAL SOEDIRMAN     |3009            |
+-----------------------------------+----------------+



#### 3b. Top 10 Universitas Paling Diminati untuk Pemilihan Jurusan Favorit IPS (yaitu: MANAJEMEN)

In [ ]:
favorit_univ_manajemen = spark.sql("""
  SELECT Nama_Universitas, COUNT(*) AS Jumlah_Pendaftar
FROM (
    SELECT university_name_x AS Nama_Universitas FROM data_ips
    UNION ALL
    SELECT university_name_y AS Nama_Universitas FROM data_ips
    WHERE data_ips.major_name_x = "MANAJEMEN" OR data_ips.major_name_y = "MANAJEMEN"
) AS combined_data
GROUP BY Nama_Universitas
ORDER BY Jumlah_Pendaftar DESC
LIMIT 10;
"""
)
favorit_univ_manajemen.limit(favorit_univ_manajemen.count()).show(truncate=False)

+--------------------------------+----------------+
|Nama_Universitas                |Jumlah_Pendaftar|
+--------------------------------+----------------+
|UNIVERSITAS INDONESIA           |4767            |
|UNIVERSITAS PADJADJARAN         |4173            |
|UNIVERSITAS BRAWIJAYA           |3847            |
|UNIVERSITAS PENDIDIKAN INDONESIA|3763            |
|UNIVERSITAS DIPONEGORO          |3734            |
|UNIVERSITAS GADJAH MADA         |3449            |
|UNIVERSITAS SEBELAS MARET       |3134            |
|UNIVERSITAS NEGERI JAKARTA      |2986            |
|UNIVERSITAS NEGERI YOGYAKARTA   |2665            |
|UNIVERSITAS NEGERI SEMARANG     |2566            |
+--------------------------------+----------------+



### 4. Informasi masing-masing universitas

- Jumlah mahasiswa baru yang diterima tahun ini (kapasitas)
- Rata-rata Skor UTBK pendaftar universitas tersebut
- Persentase penerimaan Universitas (kapasitas $\div$ jumlah pendaftar)

#### 4a. Informasi masing-masing universitas untuk segmen penjurusan IPA

In [ ]:
pendaftar_univ_ipa = spark.sql("""
  SELECT Nama_Universitas, COUNT(*) AS Jumlah_Pendaftar
FROM (
    SELECT university_name_x AS Nama_Universitas FROM data_ipa
    UNION ALL
    SELECT university_name_y AS Nama_Universitas FROM data_ipa
) AS combined_data
GROUP BY Nama_Universitas
ORDER BY Jumlah_Pendaftar DESC;
"""
).createOrReplaceTempView("pendaftar_univ_ipa")

In [ ]:
pendaftar_univ_ipa = spark.sql("""
  SELECT Nama_Universitas, COUNT(*) AS Jumlah_Pendaftar
FROM (
    SELECT university_name_x AS Nama_Universitas FROM data_ipa
    UNION ALL
    SELECT university_name_y AS Nama_Universitas FROM data_ipa
) AS combined_data
GROUP BY Nama_Universitas
ORDER BY Jumlah_Pendaftar DESC;
"""
).createOrReplaceGlobalTempView("pendaftar_univ_ipa")

In [ ]:
rata2_skor_univ_ipa = spark.sql("""
  SELECT Nama_Universitas, CAST(AVG(rata2_skor_utbk) AS INT) AS skor_utbk
FROM (
    SELECT university_name_x AS Nama_Universitas, score_utbk AS rata2_skor_utbk FROM data_ipa
    UNION ALL
    SELECT university_name_y AS Nama_Universitas, score_utbk AS rata2_skor_utbk FROM data_ipa
) AS combined_data
GROUP BY Nama_Universitas
ORDER BY skor_utbk DESC;
"""
).createOrReplaceTempView("rata2_skor_univ_ipa")

In [ ]:
test_kapasitas_univ_ipa = spark.sql("""
  SELECT Nama_Universitas, jurusan, kapasitas
FROM (
    SELECT university_name_x AS Nama_Universitas, major_name_x AS jurusan, capacity_x AS kapasitas FROM data_ipa
    UNION ALL
    SELECT university_name_y AS Nama_Universitas, major_name_y AS jurusan, capacity_y AS kapasitas FROM data_ipa
) AS combined_data
GROUP BY Nama_Universitas, jurusan, kapasitas
ORDER BY SUM(kapasitas) DESC;
"""
).createOrReplaceTempView("test_kapasitas_univ_ipa")

In [ ]:
kapasitas_univ_ipa = spark.sql("""
  SELECT Nama_Universitas, SUM(kapasitas) AS kapasitas
  FROM test_kapasitas_univ_ipa
  GROUP BY Nama_Universitas
  ORDER BY kapasitas DESC;
"""
).createOrReplaceTempView("kapasitas_univ_ipa")

In [ ]:
test_info_univ_ipa = spark.sql("""
  SELECT pendaftar_univ_ipa.Nama_Universitas, kapasitas_univ_ipa.kapasitas AS Jumlah_Mahasiswa_Baru, rata2_skor_univ_ipa.skor_utbk AS avg_skor_utbk
  FROM pendaftar_univ_ipa
  JOIN kapasitas_univ_ipa ON pendaftar_univ_ipa.Nama_Universitas = kapasitas_univ_ipa.Nama_Universitas
  JOIN rata2_skor_univ_ipa ON kapasitas_univ_ipa.Nama_Universitas = rata2_skor_univ_ipa.Nama_Universitas
  ORDER BY Jumlah_Mahasiswa_Baru DESC;
"""
).createOrReplaceGlobalTempView("test_info_univ_ipa")

In [ ]:
pendaftar_univ_ipa = spark.sql("SELECT * FROM global_temp.pendaftar_univ_ipa")
info_univ_ipa = spark.sql("SELECT * FROM global_temp.test_info_univ_ipa")

RUN SEKALI AJA

⬇⬇⬇⬇⬇

In [ ]:
info_univ_ipa = info_univ_ipa.join(pendaftar_univ_ipa, "Nama_Universitas")
info_univ_ipa = info_univ_ipa.withColumn("acceptance_rate", (info_univ_ipa.Jumlah_Mahasiswa_Baru / info_univ_ipa.Jumlah_Pendaftar * 100).cast("double"))
info_univ_ipa = info_univ_ipa.drop("Jumlah_Pendaftar")
info_univ_ipa = info_univ_ipa.orderBy(info_univ_ipa.Jumlah_Mahasiswa_Baru.desc())
info_univ_ipa.limit(info_univ_ipa.count()).show(truncate=False)

+-----------------------------------+---------------------+-------------+------------------+
|Nama_Universitas                   |Jumlah_Mahasiswa_Baru|avg_skor_utbk|acceptance_rate   |
+-----------------------------------+---------------------+-------------+------------------+
|UNIVERSITAS BRAWIJAYA              |5116                 |557          |41.83498241884046 |
|UNIVERSITAS DIPONEGORO             |3623                 |569          |35.48829464198256 |
|UNIVERSITAS GADJAH MADA            |3249                 |587          |36.25306851149297 |
|UNIVERSITAS JEMBER                 |2766                 |540          |49.810913019989194|
|UNIVERSITAS PADJADJARAN            |2661                 |559          |31.44646655636965 |
|UNIVERSITAS SRIWIJAYA              |2587                 |533          |62.74557361144798 |
|UNIVERSITAS SEBELAS MARET          |2552                 |559          |31.37062077443147 |
|UNIVERSITAS HASANUDDIN             |2522                 |525        

#### 4b. Informasi masing-masing universitas untuk segmen penjurusan IPS

In [ ]:
pendaftar_univ_ips = spark.sql("""
  SELECT Nama_Universitas, COUNT(*) AS Jumlah_Pendaftar
FROM (
    SELECT university_name_x AS Nama_Universitas FROM data_ips
    UNION ALL
    SELECT university_name_y AS Nama_Universitas FROM data_ips
) AS combined_data
GROUP BY Nama_Universitas
ORDER BY Jumlah_Pendaftar DESC;
"""
).createOrReplaceTempView("pendaftar_univ_ips")

In [ ]:
pendaftar_univ_ips = spark.sql("""
  SELECT Nama_Universitas, COUNT(*) AS Jumlah_Pendaftar
FROM (
    SELECT university_name_x AS Nama_Universitas FROM data_ips
    UNION ALL
    SELECT university_name_y AS Nama_Universitas FROM data_ips
) AS combined_data
GROUP BY Nama_Universitas
ORDER BY Jumlah_Pendaftar DESC;
"""
).createOrReplaceGlobalTempView("pendaftar_univ_ips")

In [ ]:
rata2_skor_univ_ips = spark.sql("""
  SELECT Nama_Universitas, CAST(AVG(rata2_skor_utbk) AS INT) AS skor_utbk
FROM (
    SELECT university_name_x AS Nama_Universitas, score_utbk AS rata2_skor_utbk FROM data_ips
    UNION ALL
    SELECT university_name_y AS Nama_Universitas, score_utbk AS rata2_skor_utbk FROM data_ips
) AS combined_data
GROUP BY Nama_Universitas
ORDER BY skor_utbk DESC;
"""
).createOrReplaceTempView("rata2_skor_univ_ips")

In [ ]:
test_kapasitas_univ_ips = spark.sql("""
  SELECT Nama_Universitas, jurusan, kapasitas
FROM (
    SELECT university_name_x AS Nama_Universitas, major_name_x AS jurusan, capacity_x AS kapasitas FROM data_ips
    UNION ALL
    SELECT university_name_y AS Nama_Universitas, major_name_y AS jurusan, capacity_y AS kapasitas FROM data_ips
) AS combined_data
GROUP BY Nama_Universitas, jurusan, kapasitas
ORDER BY SUM(kapasitas) DESC;
"""
).createOrReplaceTempView("test_kapasitas_univ_ips")

In [ ]:
kapasitas_univ_ips = spark.sql("""
  SELECT Nama_Universitas, SUM(kapasitas) AS kapasitas
  FROM test_kapasitas_univ_ips
  GROUP BY Nama_Universitas
  ORDER BY kapasitas DESC;
"""
).createOrReplaceTempView("kapasitas_univ_ips")

In [ ]:
test_info_univ_ips = spark.sql("""
  SELECT pendaftar_univ_ips.Nama_Universitas, kapasitas_univ_ips.kapasitas AS Jumlah_Mahasiswa_Baru, rata2_skor_univ_ips.skor_utbk AS avg_skor_utbk
  FROM pendaftar_univ_ips
  JOIN kapasitas_univ_ips ON pendaftar_univ_ips.Nama_Universitas = kapasitas_univ_ips.Nama_Universitas
  JOIN rata2_skor_univ_ips ON kapasitas_univ_ips.Nama_Universitas = rata2_skor_univ_ips.Nama_Universitas
  ORDER BY Jumlah_Mahasiswa_Baru DESC;
"""
).createOrReplaceGlobalTempView("test_info_univ_ips")

In [ ]:
pendaftar_univ_ips = spark.sql("SELECT * FROM global_temp.pendaftar_univ_ips")
info_univ_ips = spark.sql("SELECT * FROM global_temp.test_info_univ_ips")

RUN SEKALI AJA

⬇⬇⬇⬇⬇

In [ ]:
info_univ_ips = info_univ_ips.join(pendaftar_univ_ips, "Nama_Universitas")
info_univ_ips = info_univ_ips.withColumn("acceptance_rate", (info_univ_ips.Jumlah_Mahasiswa_Baru / info_univ_ips.Jumlah_Pendaftar * 100).cast("double"))
info_univ_ips = info_univ_ips.drop("Jumlah_Pendaftar")
info_univ_ips = info_univ_ips.orderBy(info_univ_ips.Jumlah_Mahasiswa_Baru.desc())
info_univ_ips.limit(info_univ_ipa.count()).show(truncate=False)

+--------------------------------+---------------------+-------------+------------------+
|Nama_Universitas                |Jumlah_Mahasiswa_Baru|avg_skor_utbk|acceptance_rate   |
+--------------------------------+---------------------+-------------+------------------+
|UNIVERSITAS PENDIDIKAN INDONESIA|3097                 |523          |44.773745843573806|
|UNIVERSITAS BRAWIJAYA           |3053                 |556          |46.187594553706504|
|UNIVERSITAS INDONESIA           |2565                 |576          |38.19237641453246 |
|UNIVERSITAS DIPONEGORO          |2555                 |569          |42.69719251336899 |
|UNIVERSITAS NEGERI MALANG       |2504                 |530          |60.17784186493631 |
|UNIVERSITAS NEGERI SEMARANG     |2424                 |533          |47.9146076299664  |
|UNIVERSITAS NEGERI SURABAYA     |2272                 |523          |64.52712297642715 |
|UNIVERSITAS AIRLANGGA           |2263                 |565          |72.346547314578   |
|UNIVERSIT

### 5. Informasi masing-masing Jurusan

- Jumlah mahasiswa baru untuk masing-masing jurusan tahun ini (kapasitas)
- Rata-rata Skor UTBK pendaftar jurusan tersebut
- Persentase penerimaan masing-masing jurusan
 (kapasitas $\div$ jumlah pendaftar)

#### 5a. Informasi masing-masing jurusan untuk segmen penjurusan IPA

In [ ]:
pendaftar_jurusan_ipa = spark.sql("""
  SELECT Nama_Jurusan, COUNT(*) AS Jumlah_Pendaftar
FROM (
    SELECT major_name_x AS Nama_Jurusan FROM data_ipa
    UNION ALL
    SELECT major_name_y AS Nama_Jurusan FROM data_ipa
) AS combined_data
GROUP BY Nama_Jurusan
ORDER BY Jumlah_Pendaftar DESC;
"""
).createOrReplaceTempView("pendaftar_jurusan_ipa")

In [ ]:
pendaftar_jurusan_ipa = spark.sql("""
  SELECT Nama_Jurusan, COUNT(*) AS Jumlah_Pendaftar
FROM (
    SELECT major_name_x AS Nama_Jurusan FROM data_ipa
    UNION ALL
    SELECT major_name_y AS Nama_Jurusan FROM data_ipa
) AS combined_data
GROUP BY Nama_Jurusan
ORDER BY Jumlah_Pendaftar DESC;
"""
).createOrReplaceGlobalTempView("pendaftar_jurusan_ipa")

In [ ]:
rata2_skor_jurusan_ipa = spark.sql("""
  SELECT Nama_Jurusan, CAST(AVG(rata2_skor_utbk) AS INT) AS skor_utbk
FROM (
    SELECT major_name_x AS Nama_Jurusan, score_utbk AS rata2_skor_utbk FROM data_ipa
    UNION ALL
    SELECT major_name_y AS Nama_Jurusan, score_utbk AS rata2_skor_utbk FROM data_ipa
) AS combined_data
GROUP BY Nama_Jurusan
ORDER BY skor_utbk DESC;
"""
).createOrReplaceTempView("rata2_skor_jurusan_ipa")

In [ ]:
test_kapasitas_jurusan_ipa = spark.sql("""
  SELECT Nama_Universitas, Nama_Jurusan, kapasitas
FROM (
    SELECT university_name_x AS Nama_Universitas, major_name_x AS Nama_Jurusan, capacity_x AS kapasitas FROM data_ipa
    UNION ALL
    SELECT university_name_y AS Nama_Universitas, major_name_y AS Nama_Jurusan, capacity_y AS kapasitas FROM data_ipa
) AS combined_data
GROUP BY Nama_Universitas, Nama_Jurusan, kapasitas
ORDER BY SUM(kapasitas) DESC;
"""
).createOrReplaceTempView("test_kapasitas_jurusan_ipa")

In [ ]:
kapasitas_jurusan_ipa = spark.sql("""
  SELECT Nama_Jurusan, SUM(kapasitas) AS kapasitas
  FROM test_kapasitas_jurusan_ipa
  GROUP BY Nama_Jurusan
  ORDER BY kapasitas DESC;
"""
).createOrReplaceTempView("kapasitas_jurusan_ipa")

In [ ]:
test_info_jurusan_ipa = spark.sql("""
  SELECT pendaftar_jurusan_ipa.Nama_Jurusan, kapasitas_jurusan_ipa.kapasitas AS Jumlah_Mahasiswa_Baru, rata2_skor_jurusan_ipa.skor_utbk AS avg_skor_utbk, pendaftar_jurusan_ipa.Jumlah_Pendaftar
  FROM pendaftar_jurusan_ipa
  JOIN kapasitas_jurusan_ipa ON pendaftar_jurusan_ipa.Nama_Jurusan = kapasitas_jurusan_ipa.Nama_Jurusan
  JOIN rata2_skor_jurusan_ipa ON kapasitas_jurusan_ipa.Nama_Jurusan = rata2_skor_jurusan_ipa.Nama_Jurusan
  ORDER BY Jumlah_Mahasiswa_Baru DESC;
"""
).createOrReplaceGlobalTempView("test_info_jurusan_ipa")

In [ ]:
pendaftar_jurusan_ipa = spark.sql("SELECT * FROM global_temp.pendaftar_univ_ipa")
info_jurusan_ipa = spark.sql("SELECT * FROM global_temp.test_info_jurusan_ipa")

RUN SEKALI AJA

⬇⬇⬇⬇⬇

In [ ]:
#info_jurusan_ipa = info_jurusan_ipa.join(pendaftar_jurusan_ipa, "Nama_Jurusan")
info_jurusan_ipa = info_jurusan_ipa.withColumn("acceptance_rate", (info_jurusan_ipa.Jumlah_Mahasiswa_Baru / info_jurusan_ipa.Jumlah_Pendaftar * 100).cast("double"))
info_jurusan_ipa = info_jurusan_ipa.drop("Jumlah_Pendaftar")
info_jurusan_ipa = info_jurusan_ipa.orderBy(info_jurusan_ipa.Jumlah_Mahasiswa_Baru.desc())
info_jurusan_ipa.limit(info_jurusan_ipa.count()).show(truncate=False)

+---------------------+---------------------+-------------+------------------+
|Nama_Jurusan         |Jumlah_Mahasiswa_Baru|avg_skor_utbk|acceptance_rate   |
+---------------------+---------------------+-------------+------------------+
|PENDIDIKAN DOKTER    |4185                 |579          |44.45034519383962 |
|TEKNIK SIPIL         |3905                 |549          |61.99396729639626 |
|AGRIBISNIS           |3458                 |529          |81.51815181518151 |
|BIOLOGI              |2958                 |539          |65.50044286979629 |
|TEKNIK ELEKTRO       |2819                 |550          |70.90040241448692 |
|TEKNIK INFORMATIKA   |2683                 |537          |54.87829822049499 |
|PETERNAKAN           |2527                 |523          |127.30478589420655|
|TEKNIK INDUSTRI      |2490                 |557          |51.46754857379082 |
|TEKNIK MESIN         |2396                 |554          |74.17956656346749 |
|KIMIA                |2357                 |552    

#### 5b. Informasi masing-masing jurusan untuk segmen penjurusan IPS

In [ ]:
pendaftar_jurusan_ips = spark.sql("""
  SELECT Nama_Jurusan, COUNT(*) AS Jumlah_Pendaftar
FROM (
    SELECT major_name_x AS Nama_Jurusan FROM data_ips
    UNION ALL
    SELECT major_name_y AS Nama_Jurusan FROM data_ips
) AS combined_data
GROUP BY Nama_Jurusan
ORDER BY Jumlah_Pendaftar DESC;
"""
).createOrReplaceTempView("pendaftar_jurusan_ips")

In [ ]:
pendaftar_jurusan_ips = spark.sql("""
  SELECT Nama_Jurusan, COUNT(*) AS Jumlah_Pendaftar
FROM (
    SELECT major_name_x AS Nama_Jurusan FROM data_ips
    UNION ALL
    SELECT major_name_y AS Nama_Jurusan FROM data_ips
) AS combined_data
GROUP BY Nama_Jurusan
ORDER BY Jumlah_Pendaftar DESC;
"""
).createOrReplaceGlobalTempView("pendaftar_jurusan_ips")

In [ ]:
rata2_skor_jurusan_ips = spark.sql("""
  SELECT Nama_Jurusan, CAST(AVG(rata2_skor_utbk) AS INT) AS skor_utbk
FROM (
    SELECT major_name_x AS Nama_Jurusan, score_utbk AS rata2_skor_utbk FROM data_ips
    UNION ALL
    SELECT major_name_y AS Nama_Jurusan, score_utbk AS rata2_skor_utbk FROM data_ips
) AS combined_data
GROUP BY Nama_Jurusan
ORDER BY skor_utbk DESC;
"""
).createOrReplaceTempView("rata2_skor_jurusan_ips")

In [ ]:
test_kapasitas_jurusan_ips = spark.sql("""
  SELECT Nama_Universitas, Nama_Jurusan, kapasitas
FROM (
    SELECT university_name_x AS Nama_Universitas, major_name_x AS Nama_Jurusan, capacity_x AS kapasitas FROM data_ips
    UNION ALL
    SELECT university_name_y AS Nama_Universitas, major_name_y AS Nama_Jurusan, capacity_y AS kapasitas FROM data_ips
) AS combined_data
GROUP BY Nama_Universitas, Nama_Jurusan, kapasitas
ORDER BY SUM(kapasitas) DESC;
"""
).createOrReplaceTempView("test_kapasitas_jurusan_ips")

In [ ]:
kapasitas_jurusan_ips = spark.sql("""
  SELECT Nama_Jurusan, SUM(kapasitas) AS kapasitas
  FROM test_kapasitas_jurusan_ips
  GROUP BY Nama_Jurusan
  ORDER BY kapasitas DESC;
"""
).createOrReplaceTempView("kapasitas_jurusan_ips")

In [ ]:
test_info_jurusan_ips = spark.sql("""
  SELECT pendaftar_jurusan_ips.Nama_Jurusan, kapasitas_jurusan_ips.kapasitas AS Jumlah_Mahasiswa_Baru, rata2_skor_jurusan_ips.skor_utbk AS avg_skor_utbk, pendaftar_jurusan_ips.Jumlah_Pendaftar
  FROM pendaftar_jurusan_ips
  JOIN kapasitas_jurusan_ips ON pendaftar_jurusan_ips.Nama_Jurusan = kapasitas_jurusan_ips.Nama_Jurusan
  JOIN rata2_skor_jurusan_ips ON kapasitas_jurusan_ips.Nama_Jurusan = rata2_skor_jurusan_ips.Nama_Jurusan
  ORDER BY Jumlah_Mahasiswa_Baru DESC;
"""
).createOrReplaceGlobalTempView("test_info_jurusan_ips")

In [ ]:
pendaftar_jurusan_ips = spark.sql("SELECT * FROM global_temp.pendaftar_univ_ips")
info_jurusan_ips = spark.sql("SELECT * FROM global_temp.test_info_jurusan_ips")

RUN SEKALI AJA

⬇⬇⬇⬇⬇

In [ ]:
#info_jurusan_ips = info_jurusan_ips.join(pendaftar_jurusan_ips, "Nama_Jurusan")
info_jurusan_ips = info_jurusan_ips.withColumn("acceptance_rate", (info_jurusan_ips.Jumlah_Mahasiswa_Baru / info_jurusan_ips.Jumlah_Pendaftar * 100).cast("double"))
info_jurusan_ips = info_jurusan_ips.drop("Jumlah_Pendaftar")
info_jurusan_ips = info_jurusan_ips.orderBy(info_jurusan_ips.Jumlah_Mahasiswa_Baru.desc())
info_jurusan_ips.limit(info_jurusan_ips.count()).show(truncate=False)

+--------------------------------------+---------------------+-------------+------------------+
|Nama_Jurusan                          |Jumlah_Mahasiswa_Baru|avg_skor_utbk|acceptance_rate   |
+--------------------------------------+---------------------+-------------+------------------+
|ILMU HUKUM                            |12246                |547          |146.74655482324746|
|MANAJEMEN                             |7646                 |540          |72.87457110179184 |
|AKUNTANSI                             |6220                 |553          |83.28869844670595 |
|ILMU KOMUNIKASI                       |3872                 |536          |48.77802973041068 |
|PSIKOLOGI                             |2840                 |545          |50.46197583511016 |
|EKONOMI PEMBANGUNAN                   |2804                 |544          |77.82403552595059 |
|HUKUM                                 |2415                 |565          |86.34250983196281 |
|SOSIOLOGI                             |